# Pytorch로 딥러닝 제대로 배우기-중급
## Part6-3: CNN 실습 (1)
### 목차
1. 데이터 호출(Cifar-10)
2. 모델
3. 학습
4. 모델 변경
 - Kernel Size 변경
 - Stride 변경

### (1) 데이터 호출 (Cifar-10)

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose

In [2]:
## Transform 지정 - ** 초급강좌에서 설명 **
transform = Compose(
    [ToTensor(),
     Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Batch size와 학습의 관계? 
batch_size = 64

trainset = datasets.CIFAR10(root='./data', train=True,
                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### (2) 모델

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 3)
    self.fc1 = nn.Linear(16 * 6 * 6, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    # Feature extractor
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)

    # Classifier
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    logits = self.fc3(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### Weights 살펴보기

In [4]:
for k, v in model.state_dict().items():
  print(k)
  print(v.size())

conv1.weight
torch.Size([6, 3, 5, 5])
conv1.bias
torch.Size([6])
conv2.weight
torch.Size([16, 6, 3, 3])
conv2.bias
torch.Size([16])
fc1.weight
torch.Size([120, 576])
fc1.bias
torch.Size([120])
fc2.weight
torch.Size([84, 120])
fc2.bias
torch.Size([84])
fc3.weight
torch.Size([10, 84])
fc3.bias
torch.Size([10])


### (3) 학습

#### 학습/평가 함수 정의

In [5]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [6]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### SGD

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [8]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304085  [    0/50000]
loss: 2.307589  [ 6400/50000]
loss: 2.291167  [12800/50000]
loss: 2.290960  [19200/50000]
loss: 2.290196  [25600/50000]
loss: 2.273208  [32000/50000]
loss: 2.244596  [38400/50000]
loss: 2.263277  [44800/50000]
Test Error: 
 Accuracy: 19.6%, Avg loss: 2.226839 

Epoch 2
-------------------------------
loss: 2.277741  [    0/50000]
loss: 2.206013  [ 6400/50000]
loss: 2.188326  [12800/50000]
loss: 2.155127  [19200/50000]
loss: 2.100488  [25600/50000]
loss: 2.082906  [32000/50000]
loss: 2.036437  [38400/50000]
loss: 2.150787  [44800/50000]
Test Error: 
 Accuracy: 28.0%, Avg loss: 1.997850 

Epoch 3
-------------------------------
loss: 1.886393  [    0/50000]
loss: 1.886312  [ 6400/50000]
loss: 1.869188  [12800/50000]
loss: 2.043580  [19200/50000]
loss: 2.015831  [25600/50000]
loss: 1.878670  [32000/50000]
loss: 1.911336  [38400/50000]
loss: 1.847695  [44800/50000]
Test Error: 
 Accuracy: 32.6%, Avg loss: 1.842094 

Epoc

#### Adam

In [9]:
model = Net().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.279012  [    0/50000]
loss: 2.022541  [ 6400/50000]
loss: 1.715575  [12800/50000]
loss: 1.541879  [19200/50000]
loss: 1.561928  [25600/50000]
loss: 1.335334  [32000/50000]
loss: 1.462776  [38400/50000]
loss: 1.243994  [44800/50000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 1.449257 

Epoch 2
-------------------------------
loss: 1.250525  [    0/50000]
loss: 1.408540  [ 6400/50000]
loss: 1.210467  [12800/50000]
loss: 1.240666  [19200/50000]
loss: 1.276740  [25600/50000]
loss: 1.360122  [32000/50000]
loss: 1.481017  [38400/50000]
loss: 1.527394  [44800/50000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.253398 

Epoch 3
-------------------------------
loss: 1.292487  [    0/50000]
loss: 1.404997  [ 6400/50000]
loss: 1.464624  [12800/50000]
loss: 1.073976  [19200/50000]
loss: 1.216717  [25600/50000]
loss: 1.293453  [32000/50000]
loss: 1.286347  [38400/50000]
loss: 1.280319  [44800/50000]
Test Error: 
 Accuracy: 57.9%, Avg loss: 1.180127 

Epoc

### (4) 모델변경

#### Kernel size 변경

In [11]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=7, stride=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 32, 7)

    # Classifier
    self.fc1 = nn.Linear(32 * 3 * 3, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    # Feature extractor
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)

    # Classifier
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    logits = self.fc3(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(7, 7), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 32, kernel_size=(7, 7), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
for k, v in model.state_dict().items():
  print(k)
  print(v.size())

conv1.weight
torch.Size([6, 3, 7, 7])
conv1.bias
torch.Size([6])
conv2.weight
torch.Size([32, 6, 7, 7])
conv2.bias
torch.Size([32])
fc1.weight
torch.Size([120, 288])
fc1.bias
torch.Size([120])
fc2.weight
torch.Size([84, 120])
fc2.bias
torch.Size([84])
fc3.weight
torch.Size([10, 84])
fc3.bias
torch.Size([10])


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [14]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.290455  [    0/50000]
loss: 1.872658  [ 6400/50000]
loss: 1.703831  [12800/50000]
loss: 1.624520  [19200/50000]
loss: 1.686553  [25600/50000]
loss: 1.761023  [32000/50000]
loss: 1.536194  [38400/50000]
loss: 1.371668  [44800/50000]
Test Error: 
 Accuracy: 46.2%, Avg loss: 1.479387 

Epoch 2
-------------------------------
loss: 1.388186  [    0/50000]
loss: 1.494177  [ 6400/50000]
loss: 1.473749  [12800/50000]
loss: 1.310159  [19200/50000]
loss: 1.339710  [25600/50000]
loss: 1.123898  [32000/50000]
loss: 1.408862  [38400/50000]
loss: 1.231587  [44800/50000]
Test Error: 
 Accuracy: 51.9%, Avg loss: 1.344855 

Epoch 3
-------------------------------
loss: 1.438165  [    0/50000]
loss: 1.513098  [ 6400/50000]
loss: 1.321483  [12800/50000]
loss: 1.365692  [19200/50000]
loss: 1.584100  [25600/50000]
loss: 1.145357  [32000/50000]
loss: 1.351648  [38400/50000]
loss: 1.227876  [44800/50000]
Test Error: 
 Accuracy: 53.8%, Avg loss: 1.282301 

Epoc

#### Stride 변경

In [15]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=2, stride=2)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 32, 3, stride=2)

    # Classifier
    self.fc1 = nn.Linear(32 * 1 * 1, 20)
    self.fc2 = nn.Linear(20, 10)
  
  def forward(self, x):
    # Feature extractor
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)

    # Classifier
    x = F.relu(self.fc1(x))
    logits = self.fc2(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(2, 2), stride=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 32, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=32, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
)


In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [17]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295084  [    0/50000]
loss: 2.174772  [ 6400/50000]
loss: 2.189975  [12800/50000]
loss: 1.948485  [19200/50000]
loss: 1.975184  [25600/50000]
loss: 1.997846  [32000/50000]
loss: 2.066010  [38400/50000]
loss: 1.945227  [44800/50000]
Test Error: 
 Accuracy: 30.2%, Avg loss: 1.860032 

Epoch 2
-------------------------------
loss: 1.934538  [    0/50000]
loss: 1.791634  [ 6400/50000]
loss: 1.898842  [12800/50000]
loss: 1.850933  [19200/50000]
loss: 1.872929  [25600/50000]
loss: 1.851153  [32000/50000]
loss: 1.958408  [38400/50000]
loss: 1.918700  [44800/50000]
Test Error: 
 Accuracy: 34.5%, Avg loss: 1.776463 

Epoch 3
-------------------------------
loss: 1.863081  [    0/50000]
loss: 1.912871  [ 6400/50000]
loss: 1.736016  [12800/50000]
loss: 1.764629  [19200/50000]
loss: 1.840791  [25600/50000]
loss: 1.654660  [32000/50000]
loss: 1.681261  [38400/50000]
loss: 1.744495  [44800/50000]
Test Error: 
 Accuracy: 36.9%, Avg loss: 1.739547 

Epoc